# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [166]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

from sklearn.grid_search import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [117]:
# load data from database
engine = create_engine('sqlite:///DisasterDatabase.db')
df = pd.read_sql_table('DisasterTable', con = engine)
df = df.dropna()
X = df["message"]
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [118]:
def tokenize(text):
    # Normilize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize
    tokens = nltk.word_tokenize(text)
    # remove stop words
    tokens = [tok for tok in tokens if tok not in stopwords.words('english')]
    # initiate lemmatizer
    lemmatizer = nltk.WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(tok) for tok in tokens]
    return cleaned_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [119]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [120]:
import time
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [150]:
target_cols = df.columns[4:].tolist()
y_pred = pd.DataFrame(pipeline.predict(X_test), columns= target_cols)


In [177]:
def print_score(y_test, y_pred, target_cols, score = "classification_report"):
    """function to plot the prediction score with three metrics"""
    for col in target_cols:
        print("score of {} output".format(col))
        if score not in ['classification_report', 'accuracy', 'F1_score']:
            print("pleaseselect specify score as one the following metrics: classification_report, accuracy, F1_score ")
            break;
        elif score == "classification_report":
            print(classification_report(y_test[col], y_pred[col], target_names= col))
        elif score == "accuracy":
            print(accuracy_score(y_test[col], y_pred[col]))
        elif score == "F1_score":
            print(f1_score(y_test[col], y_pred[col], average='weighted'))

            
print_score(y_test, y_pred, target_cols, score = "classification_report")

score of related output
             precision    recall  f1-score   support

          r       0.38      0.27      0.31       825
          e       0.68      0.79      0.73      1688
          l       0.00      0.00      0.00        43

avg / total       0.57      0.61      0.58      2556

score of request output
             precision    recall  f1-score   support

          r       0.66      0.90      0.76      1640
          e       0.46      0.15      0.23       916

avg / total       0.59      0.63      0.57      2556

score of offer output
             precision    recall  f1-score   support

          o       1.00      1.00      1.00      2549
          f       0.00      0.00      0.00         7

avg / total       0.99      1.00      1.00      2556

score of aid_related output
             precision    recall  f1-score   support

          a       0.62      0.84      0.71      1560
          i       0.45      0.21      0.28       996

avg / total       0.55      0.59      0.55 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 11
  .format(len(labels), len(target_names

### 6. Improve your model
Use grid search to find better parameters. 

In [182]:
list(df.columns[4:])


['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [165]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fc8ca97cf28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [172]:
parameters = {
        'clf__estimator__n_estimators': [20, 50],
        'clf__estimator__max_features': [10, 'auto'],
}

cv = GridSearchCV(pipeline, param_grid=parameters)
start = time.time()
cv.fit(X_train, y_train)
print("execution time {}".format(time.time()-start))

execution time 798.0373060703278


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [173]:
cv_y_pred = pd.DataFrame(cv.predict(X_test), columns= target_cols)
print_score(y_test, cv_y_pred, target_cols, score = "F1_score")

score of related output
0.565813540529
score of request output
0.569128705273
score of offer output
0.995893896407
score of aid_related output
0.549973699019
score of medical_help output
0.9140087188
score of medical_products output
0.947498611742
score of search_and_rescue output
0.967840019706
score of security output
0.981260088968
score of military output
0.993549241166
score of child_alone output
1.0
score of water output
0.883244625464
score of food output
0.779764375912
score of shelter output
0.839351268316
score of clothing output
0.988667977139
score of money output
0.980091480571
score of missing_people output
0.985352672574
score of refugees output
0.973086401905
score of death output
0.959110296669
score of other_aid output
0.782459588037
score of infrastructure_related output
0.951748144662
score of transport output
0.971141259957
score of buildings output
0.940734690077
score of electricity output
0.992181720286
score of tools output
0.995893896407
score of hospitals out

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [174]:
pipeline_kn = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])
# fit the pipeline
pipeline_kn.fit(X_train, y_train)
# predict
y_pred_kn = pd.DataFrame(pipeline_kn.predict(X_test), columns= target_cols)
# plot results
print_score(y_test, y_pred_kn, target_cols, score = "F1_score")

score of related output
0.567077884759
score of request output
0.539901755392
score of offer output
0.995893896407
score of aid_related output
0.509676788761
score of medical_help output
0.913813235155
score of medical_products output
0.947498611742
score of search_and_rescue output
0.967840019706
score of security output
0.981260088968
score of military output
0.993549241166
score of child_alone output
1.0
score of water output
0.88458302001
score of food output
0.779498254382
score of shelter output
0.839201391926
score of clothing output
0.988863630986
score of money output
0.980091480571
score of missing_people output
0.985352672574
score of refugees output
0.973086401905
score of death output
0.959110296669
score of other_aid output
0.784036255392
score of infrastructure_related output
0.951943827522
score of transport output
0.971336898429
score of buildings output
0.940930345881
score of electricity output
0.992377375539
score of tools output
0.995893896407
score of hospitals ou

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [175]:
from sklearn.externals import joblib
joblib.dump(cv.best_estimator_, 'disaster_clf.pkl')

['disaster_clf.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.